In [1]:
#Introduction

In [2]:
"""
This example looks at the Kaggle Credit Card Fraud Detection dataset to demonstrate how to train a classification model on data with highly imbalanced classes.
"""

'\nThis example looks at the Kaggle Credit Card Fraud Detection dataset to demonstrate how to train a classification model on data with highly imbalanced classes.\n'

In [3]:
#import the libraries that we will use

In [2]:
import csv
import numpy as np

In [3]:
#First, vectorize the CSV data

In [4]:
# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "C:/Users/salma/Downloads/creditcard.csv/creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


In [5]:
#Prepare a validation set

In [6]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


In [7]:
#Analyze class imbalance in the targets

In [8]:
counts = np.bincount(train_targets[:, 0])
print(
    "Number of positive samples in training data: {} ({:.2f}% of total)".format(
        counts[1], 100 * float(counts[1]) / len(train_targets)
    )
)

weight_for_0 = 1.0 / counts[0]
weight_for_1 = 1.0 / counts[1]

Number of positive samples in training data: 417 (0.18% of total)


In [9]:
#Normalize the data using training set statistics

In [10]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

In [11]:
#Build a binary classification model

In [12]:
from tensorflow import keras
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_addons as tfa

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               7936      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 139,777
Trainable params: 139,777
Non-trai

In [13]:
#Train the model with class_weight argument

In [14]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
    class_weight=class_weight,
)

Epoch 1/30
112/112 - 8s - loss: 2.2897e-06 - fn: 50.0000 - fp: 19515.0000 - tn: 207914.0000 - tp: 367.0000 - precision: 0.0185 - recall: 0.8801 - val_loss: 0.1064 - val_fn: 10.0000 - val_fp: 752.0000 - val_tn: 56134.0000 - val_tp: 65.0000 - val_precision: 0.0796 - val_recall: 0.8667 - 8s/epoch - 72ms/step
Epoch 2/30
112/112 - 5s - loss: 1.4127e-06 - fn: 39.0000 - fp: 6193.0000 - tn: 221236.0000 - tp: 378.0000 - precision: 0.0575 - recall: 0.9065 - val_loss: 0.0974 - val_fn: 9.0000 - val_fp: 1102.0000 - val_tn: 55784.0000 - val_tp: 66.0000 - val_precision: 0.0565 - val_recall: 0.8800 - 5s/epoch - 44ms/step
Epoch 3/30
112/112 - 4s - loss: 1.1668e-06 - fn: 24.0000 - fp: 7316.0000 - tn: 220113.0000 - tp: 393.0000 - precision: 0.0510 - recall: 0.9424 - val_loss: 0.0327 - val_fn: 10.0000 - val_fp: 396.0000 - val_tn: 56490.0000 - val_tp: 65.0000 - val_precision: 0.1410 - val_recall: 0.8667 - 4s/epoch - 39ms/step
Epoch 4/30
112/112 - 4s - loss: 9.9817e-07 - fn: 27.0000 - fp: 6580.0000 - tn: 22

Epoch 28/30
112/112 - 5s - loss: 4.3552e-07 - fn: 3.0000 - fp: 4067.0000 - tn: 223362.0000 - tp: 414.0000 - precision: 0.0924 - recall: 0.9928 - val_loss: 0.1918 - val_fn: 5.0000 - val_fp: 3408.0000 - val_tn: 53478.0000 - val_tp: 70.0000 - val_precision: 0.0201 - val_recall: 0.9333 - 5s/epoch - 42ms/step
Epoch 29/30
112/112 - 5s - loss: 7.9831e-07 - fn: 8.0000 - fp: 7585.0000 - tn: 219844.0000 - tp: 409.0000 - precision: 0.0512 - recall: 0.9808 - val_loss: 0.0460 - val_fn: 10.0000 - val_fp: 449.0000 - val_tn: 56437.0000 - val_tp: 65.0000 - val_precision: 0.1265 - val_recall: 0.8667 - 5s/epoch - 41ms/step
Epoch 30/30
112/112 - 6s - loss: 5.0937e-07 - fn: 4.0000 - fp: 5019.0000 - tn: 222410.0000 - tp: 413.0000 - precision: 0.0760 - recall: 0.9904 - val_loss: 0.0294 - val_fn: 10.0000 - val_fp: 638.0000 - val_tn: 56248.0000 - val_tp: 65.0000 - val_precision: 0.0925 - val_recall: 0.8667 - 6s/epoch - 50ms/step


In [15]:
#Conclusions

In [16]:
"""
At the end of training, out of 56,961 validation transactions, we are:

1- Correctly identifying 66 of them as fraudulent
2- Missing 9 fraudulent transactions
3- At the cost of incorrectly flagging 441 legitimate transactions

In the real world, one would put an even higher weight on class 1, so as to reflect that False Negatives are more costly than False Positives.

"""

'\nAt the end of training, out of 56,961 validation transactions, we are:\n\n1- Correctly identifying 66 of them as fraudulent\n2- Missing 9 fraudulent transactions\n3- At the cost of incorrectly flagging 441 legitimate transactions\n\nIn the real world, one would put an even higher weight on class 1, so as to reflect that False Negatives are more costly than False Positives.\n\n'